# MODEL ARCHITECTURE

In [35]:
import os
print(os.getcwd())

/sfs/weka/scratch/ssb3vk/RL


In [36]:
import torch
from torch import nn
import numpy as np

class DuelingQNetworkConv(nn.Module):
    def __init__(self, action_size=12):
        super(DuelingQNetworkConv, self).__init__()

        # Convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(5, 5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=2),
        )

        # Calculate the size of the flattened feature vector after the conv layers
        # Here we assume an input of shape (1, 21, 11, 16) because Conv3d expects a batch dimension and a channel dimension
        with torch.no_grad():
            self.flattened_size = self._get_conv_output((1, 1, 21, 11, 16))

        # Fully connected layers
        self.fc_input = nn.Linear(self.flattened_size, 512)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(512, 512)

        # Dueling branches
        self.state_value = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)  # Outputs V(s)
        )
        self.action_value = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, action_size)  # Outputs A(s, a) for each action
        )

    def forward(self, obs: torch.Tensor):
        # Convolutional layers
        conv_out = self.conv_layers(obs)
        conv_out = conv_out.view(-1, self.flattened_size)  # Flatten the output for the fully connected layers

        # Fully connected layers
        h = self.relu(self.fc_input(conv_out))
        h = self.relu(self.fc1(h))

        # Compute state value and action advantage values
        state_value = self.state_value(h)
        action_value = self.action_value(h)
        action_score_centered = action_value - action_value.mean(dim=1, keepdim=True)

        # Combine state value and centered action values to get Q values
        q = state_value + action_score_centered

        return q

    def _get_conv_output(self, shape):
        input = torch.rand(shape)
        output = self.conv_layers(input)
        return int(np.prod(output.size()))


# Reading in Data and Training Models

In [37]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm  # for progress bars

def map_labels(labels):
    # Map label 17 to 12 (omitted range 12-16 because there is no trading in our simplfied version)
    labels = labels.apply(lambda x: 12 if x == 17 else x)
    return labels - 1 # Zero index them

def train_on_chunk(data_chunk, action_chunk):
   # Get the ACTION_TYPE column, as that is the one we are trying to predict
    labels = action_chunk.iloc[:, 1]
    non_zero_indices = labels != 0

    # Apply the same filter to both the data_chunk and labels to ensure they align
    filtered_data_chunk = data_chunk[non_zero_indices]
    filtered_labels = labels[non_zero_indices]
    filtered_labels = map_labels(filtered_labels) # Change 17s to 12s

    # Reshape so that convolutions can be applied
    reshaped_tensors = []
    for index, row in filtered_data_chunk.iterrows():
        tensor = torch.tensor(row.values, dtype=torch.float32).reshape((21, 11, 16))
        reshaped_tensors.append(tensor)

    X = torch.stack(reshaped_tensors).unsqueeze(1).to(device)  # Move to the specified device
    y = torch.tensor(filtered_labels.values, dtype=torch.long).to(device)  # Move to the specified device

    if X.shape[0] == 0:
        print("No samples left after filtering.")
        return

    dataset = TensorDataset(X, y)
    data_loader = DataLoader(dataset, batch_size=128, shuffle=True)

    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    avg_loss = running_loss / len(data_loader)
    accuracy = correct_predictions / total_predictions

    return avg_loss, accuracy

In [38]:
def train_model_from_files(board_tensors_path, actions_path):
    """
    Train the model using the specified board_tensors and actions files.
    """
    num_epochs = 100
    
    epoch_avg_loss = []
    epoch_avg_acc = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        epoch_losses = []
        epoch_accuracies = []

        data_tensor = pd.read_csv(board_tensors_path, compression='gzip', chunksize=10000)
        action_tensor = pd.read_csv(actions_path, compression='gzip', chunksize=10000)

        for data_chunk, action_chunk in tqdm(zip(data_tensor, action_tensor), desc="Processing chunks", total=10000 // data_tensor.chunksize):
            avg_loss, accuracy = train_on_chunk(data_chunk, action_chunk)
            if avg_loss is not None:  # Ensure chunk had data to process
                epoch_losses.append(avg_loss)
                epoch_accuracies.append(accuracy)

        if epoch_losses:  # Ensure there were processed chunks
            print(f"Average Loss: {np.mean(epoch_losses):.4f}")
            print(f"Average Accuracy: {np.mean(epoch_accuracies):.4f}")
            
        epoch_avg_loss.append(np.mean(epoch_losses))
        epoch_avg_acc.append(np.mean(epoch_accuracies))
            
        if len(epoch_avg_acc) >= 5: 
            if not epoch_avg_acc[-1] > epoch_avg_acc[-5] + 0.001:
                print(f"Ending batch's epoch's early, epoch_avg_acc[-1] = {epoch_avg_acc[-1]:.4f} <= epoch_avg_acc[-5] = {epoch_avg_acc[-5]:.4f}")

In [39]:
import os
def process_directory(base_dir):
    """
    Process each subdirectory within base_dir, assuming each contains
    'board_tensors.csv.gzip' and 'actions.csv.gzip' files.
    """
    # List all subdirectories in the base directory
    subdirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    for subdir in subdirs:
        board_tensors_path = os.path.join(subdir, 'board_tensors.csv.gzip')
        actions_path = os.path.join(subdir, 'actions.csv.gzip')

        # Ensure both files exist before proceeding
        if os.path.exists(board_tensors_path) and os.path.exists(actions_path):
            print(f"Processing directory: {subdir}")
            train_model_from_files(board_tensors_path, actions_path)


In [ ]:
import pickle
# Overall run
model = DuelingQNetworkConv(action_size=12)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss() # Cross entropy loss for classification 0-11

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)  # Move the model to the specified device before training starts


base_dir = './data'
process_directory(base_dir)

model.to('cpu')  # Move the model to CPU

# Save the model to a file
model_path = 'saved_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

Using device: cuda
Processing directory: ./data/2024-03-29-03-00-22
Epoch 1/100


Processing chunks: 21it [01:12,  3.43s/it]                      


Average Loss: 1.4229
Average Accuracy: 0.5382
Epoch 2/100


Processing chunks: 21it [01:12,  3.43s/it]                      


Average Loss: 1.3789
Average Accuracy: 0.5573
Epoch 3/100


...

Processing chunks: 21it [01:12,  3.45s/it]                      


Average Loss: 1.1501
Average Accuracy: 0.5791
Epoch 67/100


Processing chunks: 21it [01:12,  3.45s/it]                      


Average Loss: 1.1469
Average Accuracy: 0.5793
Epoch 68/100


Processing chunks: 19it [01:07,  3.51s/it]                      